## Weighted Regression in `python`                                   :jupyter:



The fact that $T$ and $u$ are &ldquo;independent&rdquo; (or at least
orthogonal) variables means that if we want to compute a
&ldquo;classical&rdquo; regression we&rsquo;d do it something like this:



##### Define independent random variables



In [1]:
%matplotlib inline
import numpy as np
from scipy.stats import multivariate_normal

k = 3 # Number of observables in T

mu = [0]*k
Sigma=[[1,0.5,0],
       [0.5,2,0],
       [0,0,3]]

T = multivariate_normal(mu,Sigma)

u = multivariate_normal(cov=0.2)

##### Define `X`



Recall that $X$ can depend on $T$ and $u$.  This dependence needn&rsquo;t be
linear!  For example, suppose $X=T^3D + u$, where $D$ is an
$\ell\times k$ matrix.



##### Construct Sample



To construct a sample of observables $(y,X,T)$ we just use the regression equation,
      plus an assumption about the value of $\beta$:



In [1]:
beta = [1/2,1]

D = np.random.random(size=(3,2)) # Generate random 3x2 matrix

N=1000 # Sample size

# Now: Transform rvs into a sample
T = T.rvs(N)

u = u.rvs(N) # Replace u with a sample

X = (T**3)@D  # Note use of ** operator for exponentiation

y = X@beta + u # Note use of @ operator for matrix multiplication

##### Turn to estimation



So, we now have data on *realizations* $(y,X,T)$.  Now forget
     that we know $\beta$ and let&rsquo;s estimate it, using weighted least
     squares.  As a numerical matter it&rsquo;s better to avoid explicitly
     inverting the $(T^T X)$ matrix; instead we can solve the &ldquo;normal&rdquo;
     equations

\begin{align*}
   T'y &= T' X b + T' u\\
   \mbox{E}(T'u) = 0
\end{align*}



##### Numerical solution



In the classical case we were trying to solve a linear system that
 took the form $Ab=0$, with $A$ a square matrix.  In the present case
 we&rsquo;re also trying to solve a linear system, but with a matrix $A$
 that may have more rows than columns.  Provided the rows are linearly
 independent, this implies that we have an **overidentified** system of
 equations.  We&rsquo;ll return to the implications of this later, but for
 now this also calls for a different numerical approach, using
 `np.linalg.lstsq` instead of `np.linalg.solve`.



In [1]:
from scipy.linalg import inv, sqrtm

b = np.linalg.lstsq(T.T@X,T.T@y,rcond=None)[0] # lstsq returns several results

e = y - X@b

print(b)

TXplus = np.linalg.pinv(T.T@X) # Moore-Penrose pseudo-inverse

# Covariance matrix of b
vb = e.var()*TXplus@T.T@T@TXplus.T  # u is known to be homoskedastic

print(vb)